# FInDe

## Availables banks
- Cajamar
- Openbank
- Unicaja

## Fist step
Put your csv or xls files inside ```accounts/current/<bank_name>/file``` and ```accounts/savings/<bank_name>/file```

## Requisites

In [ ]:
!pip install plotly

## Code

### Imports

In [ ]:
import plotly as py
import plotly.graph_objs as go

import numpy as np
import datetime as dt
import os.path
import xlrd
import pandas as pd

py.offline.init_notebook_mode(connected=True)

### Constant and settings

In [ ]:
CURRENT = "current"
SAVING = "saving"
ACCOUNTTYPES  = [CURRENT, SAVING]
SUPPORTED_BANKS = {
    "unicaja": "Unicaja",
    "cajamar": "Cajamar",
    "openbank": "Openbank"
}

### Process CSV and XLS

In [ ]:
def processCSV(file, splitToken, bank): #TODO Move to pandas
    fecha = []
    saldo = []
    first = True
    
    if bank == 'cajamar':
        with open(file) as f:
            for line in f:
                if first is False:
                    tokens = line.split(splitToken)
                    fecha.append(dt.datetime.strptime(tokens[0], '%d/%m/%Y').date())
                    saldo.append(float(tokens[len(tokens)-1][:-1].replace('.', '').replace(',', '.')))
                first = False

    return (saldo, fecha)

In [ ]:
def processXLS(file, bank):   
    balance = []
    date = []
    movements = []
    
    if bank == 'unicaja':
        book = xlrd.open_workbook(file)
        first_sheet = book.sheet_by_index(0)
        date_int = first_sheet.col_values(0)[5:]
        balance = first_sheet.col_values(5)[5:]
        movements = first_sheet.col_values(3)[5:]
        date = [xlrd.xldate_as_datetime(date_int[i], book.datemode).date().isoformat() for i in xrange(0, len(date_int))]
    
    if bank == 'openbank':
        df = pd.read_html(file)
        df = df[0].dropna(axis=0, thresh=4)
        date_str = df.iloc[1:,1]
        date = [dt.datetime.strptime(x, '%d/%m/%Y').date().isoformat() for x in date_str]
        balance_str = df.iloc[1:,9]
        balance = [float((x[:-2] + ',' + x[-2:]).replace('.','').replace(',','.')) for x in balance_str]
        movements_str = df.iloc[1:,7]
        movements = [float((x[:-2] + ',' + x[-2:]).replace('.','').replace(',','.')) for x in movements_str]

    return (balance, date, movements)

### Load files

In [ ]:
def sortDataFiles(bankdata):
    for accountType in ACCOUNTTYPES:
        for bankname in bankdata[accountType]:
            if "movements" in bankdata[accountType][bankname]:
                (bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"], bankdata[accountType][bankname]["movements"]) = zip(*sorted(zip(bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"], bankdata[accountType][bankname]["movements"])))
            else:
                (bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"]) = zip(*sorted(zip(bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"])))
            
    return bankdata

In [ ]:
def loadDataFiles():    
    print("Data files found:")
    bankdata = {}
    for accountType in ACCOUNTTYPES:
        path = "accounts/{}/".format(accountType)
        directories = [f for f in os.listdir(path) if os.path.isdir(path+f)]
        bankdata[accountType] = {}
        for bankname in directories:
            bankdata[accountType][bankname] = {"date":[], "balance":[], "movements":[]}
            files = [os.path.join(path+bankname,f) for f in os.listdir(path+bankname)]
            #files.sort(key=lambda x: os.path.getmtime(x))
            for datafile in files:
                extension = os.path.splitext(datafile)[1]
                if extension == ".xls":
                    print("{} - {}".format(bankname, datafile))
                    (balance, date, movements) = processXLS(datafile, bankname)
                    bankdata[accountType][bankname]["balance"].extend(balance)
                    bankdata[accountType][bankname]["date"].extend(date)
                    bankdata[accountType][bankname]["movements"].extend(movements)
                if extension == ".csv":
                    print("{} - {}".format(bankname, datafile))
                    (balance, date) = processCSV(datafile, '\t', bankname)
                    bankdata[accountType][bankname]["balance"].extend(balance)
                    bankdata[accountType][bankname]["date"].extend(date)
    
    return sortDataFiles(bankdata)

### Traces

In [ ]:
def buildTraces(bankdata):
    data = []
    
    for accountType in ACCOUNTTYPES: # Account type
        for bank in bankdata[accountType]: # Bank name
            dates = np.array(bankdata[accountType][bank]['date'])
            balances = np.array(bankdata[accountType][bank]['balance'])
            
            bankName = '(' + accountType + ') ' + SUPPORTED_BANKS[bank]

            trace_main = go.Scatter(
                x = dates,
                y = balances,
                name = bankName + ': Saldo ' + str(format(balances[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
                #line = dict(
                #    color = 'green'
                #),
                mode = 'lines'
            )
            data.append(trace_main)

            trace_max = go.Scatter(
                x = dates,
                y = [balances.max() for f in dates],
                name = bankName + ': Saldo máximo',
                #visible = 'legendonly',
                #hoverinfo = 'name',
                line = dict(
                    #color = 'cyan',
                    width = 4,
                    dash = 'dot'
                )
            )
            data.append(trace_max)

            trace_mean = go.Scatter(
                x = dates,
                y = [balances.mean() for f in dates],
                name = bankName + ': Saldo medio',
                #hoverinfo = 'none',
                line = dict(
                    #color = 'magenta',
                    width = 4,
                    dash = 'dashdot'
                )
            )
            data.append(trace_mean)

            trace_min = go.Scatter(
                x = dates,
                y = [balances.min() for f in dates],
                name = bankName + ': Saldo mínimo',
                line = dict(
                    #color = 'red',
                    width = 4,
                    dash = 'dot'
                )
            )
            data.append(trace_min)
        
    return data

In [ ]:
def buildPig(bankdata):
    banks = []
    amount = []
    
    for bank in bankdata[SAVING]: # Bank name
        balances = np.array(bankdata[SAVING][bank]['balance'])
        
        banks.append(SUPPORTED_BANKS[bank])
        amount.append(balances[-1])

    trace = go.Bar(
        x = banks,
        y = amount,
        name = 'Hucha'
    )
        
    return [trace]

In [ ]:
def buildProfit(bankdata):
    data = []
    
    for bank in bankdata[CURRENT]:
        date = bankdata[CURRENT][bank]['date']
        movements = bankdata[CURRENT][bank]['movements']
        pDay = date[0][:7]
        pValue = 0
        months = []
        profit = []
        for cDay, cValue in zip(date, movements):
            pValue += float(cValue)
            if cDay[:7] != pDay:
                months.append(pDay)
                profit.append(pValue)
                pDay = cDay[:7]
                pValue = 0
        months.append(pDay)
        profit.append(pValue)
        
    
        trace = go.Bar(
            x = months,
            y = profit,
            name = "Profit for {}".format(bank)
        )
        data.append(trace)
        
    return data

### Min/Max dates

In [ ]:
def getIntervalDates(bankdata):
    return (0, 0) #TODO

### Plot

In [ ]:
bankdata = loadDataFiles() # Load files
data = buildTraces(bankdata) # Build traces from data
(minDate, maxDate) = getIntervalDates(bankdata)

# Plot charts
layout = go.Layout(title = 'Saldo ' + str(minDate) + ' / ' + str(maxDate),
              xaxis = dict(title = 'Fecha'),
              yaxis = dict(title = 'Saldo (€)'),
              showlegend = True
)

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='styled-line')

# Hucha
data = buildPig(bankdata)
layout = go.Layout(
    title = 'Hucha',
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig, filename='stacked-bar')

data = buildProfit(bankdata)
layout = go.Layout(
    title = 'Monthly Profit',
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig, filename='grouped-bar')

'''
fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout'].update(height=600, width=800, title='i <3 annotations and subplots')
py.offline.iplot(fig, filename='simple-subplot-with-annotations')
'''
